In [ ]:
import spyrmsd
from spyrmsd import io, rmsd
from spyrmsd.molecule import Molecule
import numpy as np
from rdkit import Chem
import pandas as pd

In [6]:
vina_suppl = Chem.SDMolSupplier('1443_vina.sdf', sanitize=False, removeHs=True)
glide_suppl = Chem.SDMolSupplier('1443_glide.sdf', removeHs=True)
output_file = "./rmsd_1443_select.csv"

In [3]:
df_map = pd.read_csv('./merge_1443.csv')

In [ ]:
vina_mols = {}
suppl_vina = Chem.SDMolSupplier('1443_vina.sdf', sanitize=False, removeHs=True)
for mol in suppl_vina:
    if mol is not None:
        name = mol.GetProp('_Name')
        vina_mols[name] = mol

glide_mols = {}
suppl_glide = Chem.SDMolSupplier('1443_glide.sdf', removeHs=True)
for mol in suppl_glide:
    if mol is not None:
        name = mol.GetProp('_Name')
        glide_mols[name] = mol

if len(vina_mols) != len(glide_mols):
    print("Warning: The number of molecules in Vina and Glide files do not match.")
    
matched_molecules = {}
missing_molecules = []

for index, row in df_map.iterrows():
    std_smiles = row['standardized_smiles']
    title_vina = row['title_vina']+'.pdbqt'
    title_glide = row['title_glide']
    
    if title_vina in vina_mols and title_glide in glide_mols:
        matched_molecules[std_smiles] = {
            'title_vina': title_vina,
            'title_glide': title_glide,
            'mol_vina': vina_mols[title_vina],
            'mol_glide': glide_mols[title_glide]
        }
    else:
        missing_info = []
        if title_vina not in vina_mols:
            missing_info.append(f"Vina: {title_vina}")
        if title_glide not in glide_mols:
            missing_info.append(f"Glide: {title_glide}")
        missing_molecules.append({
            'smiles': std_smiles,
            'missing': ", ".join(missing_info)
        })

matched_molecules


In [ ]:
for cid in matched_molecules:
    ref = Molecule.from_rdkit(matched_molecules[cid]['mol_vina'])
    coords_ref = ref.coordinates
    anum_ref = ref.atomicnums
    adj_ref = ref.adjacency_matrix
    mol = Molecule.from_rdkit(matched_molecules[cid]['mol_glide'])
    coords = mol.coordinates
    anum = mol.atomicnums
    adj = mol.adjacency_matrix
    if coords_ref.shape[0] == coords.shape[0]:
        RMSD = rmsd.symmrmsd(coords_ref, coords, anum_ref, anum, adj_ref, adj)
        matched_molecules[cid]['RMSD'] = RMSD
    else:
        print(f"Unmatched atom count in {cid}: Vina({coords_ref.shape[0]}) vs Glide({coords.shape[0]})")
matched_molecules

In [ ]:
output_data = []
for cid, data in matched_molecules.items():
    output_data.append({
            'standardized_smiles': cid,
            'rmsd': data['RMSD']
        })
output_df = pd.DataFrame(output_data, columns = ['standardized_smiles','rmsd'])
output_df.sort_values(by = 'rmsd')
df = output_df.merge(df_map, on='standardized_smiles', how='left')
df

In [10]:
rmsd_mol = df[df['rmsd']<2.51].copy()
rmsd_mol.to_csv(output_file, index = False)

In [12]:
rmsd_mol['title_vina'].to_csv("./1443_rmsd_vinalist.txt", index = False, header = False)
rmsd_mol['title_glide'].to_csv("./1443_rmsd_glidelist.txt", index = False, header = False)